<a href="https://colab.research.google.com/github/brunatoloti/data-viz/blob/main/Visualiza%C3%A7%C3%A3o_de_dados_com_python_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualização de Dados com Python Extra

In [1]:
import pandas as pd

In [2]:
idhm = pd.read_csv('/content/drive/My Drive/Curso Visualização de Dados/Dados/atlas2013_dadosbrutos_idhm_pt.csv', sep=';', decimal=',')
covid = pd.read_excel('/content/drive/My Drive/Curso Visualização de Dados/Dados/HIST_PAINEL_COVIDBR_24jul2020.xlsx')

In [3]:
#Posso fazer esse mesmo gráfico considerando os estados e não os municípios 
#(dessa forma, podemos, talvez, conseguir fazer a animação)

covid.dropna(subset=['estado'], inplace=True)

In [4]:
covid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
151,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-25,9,1777225,0,0,0,0,NaN,NaN,NaN
152,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-26,9,1777225,0,0,0,0,NaN,NaN,NaN
153,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-27,9,1777225,0,0,0,0,NaN,NaN,NaN
154,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-28,9,1777225,0,0,0,0,NaN,NaN,NaN
155,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-29,9,1777225,0,0,0,0,NaN,NaN,NaN


In [5]:
idhm.head()

,ANO,UF,Codmun6,Codmun7,Município,IDHM,IDHM_E,IDHM_L,IDHM_R
0,2010,11,110001,1100015,ALTA FLORESTA D'OESTE,0.641,0.526,0.763,0.657
1,2010,11,110002,1100023,ARIQUEMES,0.702,0.600,0.806,0.716
2,2010,11,110003,1100031,CABIXI,0.650,0.559,0.757,0.650
3,2010,11,110004,1100049,CACOAL,0.718,0.620,0.821,0.727
4,2010,11,110005,1100056,CEREJEIRAS,0.692,0.602,0.799,0.688


In [6]:
covid = covid.groupby(by=['estado', 'data', 'coduf', 'regiao'])[['casosAcumulado', 'obitosAcumulado']].sum()

In [7]:
covid = covid.reset_index()

In [8]:
#Criando a coluna de letalidade no dataframe de covid.

covid['letalidade'] = (covid['obitosAcumulado']/covid['casosAcumulado']).fillna(0)

In [9]:
#Criando uma coluna de conversão de data

covid['format_data'] = covid['data'].dt.strftime('%d-%m')

In [10]:
covid.head()

,estado,data,coduf,regiao,casosAcumulado,obitosAcumulado,letalidade,format_data
0,AC,2020-02-25,12,Norte,0,0,0.0,25-02
1,AC,2020-02-26,12,Norte,0,0,0.0,26-02
2,AC,2020-02-27,12,Norte,0,0,0.0,27-02
3,AC,2020-02-28,12,Norte,0,0,0.0,28-02
4,AC,2020-02-29,12,Norte,0,0,0.0,29-02


In [11]:
covid_estados = covid.merge(idhm, left_on='coduf', right_on='UF')

In [12]:
covid_estados.head()

,estado,data,coduf,regiao,casosAcumulado,obitosAcumulado,letalidade,format_data,ANO,UF,Codmun6,Codmun7,Município,IDHM,IDHM_E,IDHM_L,IDHM_R
0,AC,2020-02-25,12,Norte,0,0,0.0,25-02,2010,12,120001,1200013,ACRELÂNDIA,0.604,0.466,0.808,0.584
1,AC,2020-02-25,12,Norte,0,0,0.0,25-02,2010,12,120005,1200054,ASSIS BRASIL,0.588,0.456,0.770,0.578
2,AC,2020-02-25,12,Norte,0,0,0.0,25-02,2010,12,120010,1200104,BRASILÉIA,0.614,0.485,0.770,0.619
3,AC,2020-02-25,12,Norte,0,0,0.0,25-02,2010,12,120013,1200138,BUJARI,0.589,0.439,0.772,0.603
4,AC,2020-02-25,12,Norte,0,0,0.0,25-02,2010,12,120017,1200179,CAPIXABA,0.575,0.398,0.794,0.601


In [13]:
covid_estados = covid_estados.sort_values(by='data')

In [14]:
covid_estados.describe()

,coduf,casosAcumulado,obitosAcumulado,letalidade,ANO,UF,Codmun6,Codmun7,IDHM,IDHM_E,IDHM_L,IDHM_R
count,840315.000000,840315.000000,840315.000000,840315.000000,840315.0,840315.000000,840315.000000,8.403150e+05,840315.000000,840315.000000,840315.000000,840315.000000
mean,32.372327,56863.633704,2567.413596,0.032017,2010.0,32.372327,325304.894160,3.253053e+06,0.659157,0.559094,0.801564,0.642873
std,9.829080,123411.153879,6316.381781,0.027085,0.0,9.829080,98443.360595,9.844336e+05,0.071991,0.093319,0.044677,0.080654
min,11.000000,0.000000,0.000000,0.000000,2010.0,11.000000,110001.000000,1.100015e+06,0.418000,0.207000,0.672000,0.400000
25%,25.000000,320.000000,8.000000,0.014255,2010.0,25.000000,251210.000000,2.512101e+06,0.599000,0.490000,0.769000,0.572000
50%,31.000000,7500.000000,278.000000,0.027778,2010.0,31.000000,314620.000000,3.146206e+06,0.665000,0.560000,0.808000,0.654000
75%,41.000000,59838.000000,1818.000000,0.044628,2010.0,41.000000,411900.000000,4.119004e+06,0.718000,0.631000,0.836000,0.707000
max,53.000000,926436.000000,42412.000000,0.222222,2010.0,53.000000,530010.000000,5.300108e+06,0.862000,0.825000,0.894000,0.891000


In [15]:
#Não vamos conseguir fazer a animação, devido ao poder de processamento do google colab
#Portanto, pegando os acumulados para a última data do conjunto de dados

import plotly.express as px

fig_scatter = px.scatter(covid_estados[covid_estados.data==pd.Timestamp('2020-07-24')], 
                         x="casosAcumulado", y="letalidade", 
                         size="obitosAcumulado", color="IDHM",
                         hover_name="estado", size_max=60, facet_col="regiao",
                         range_y=[-0.01,0.2], range_x=[-300000,2000000])
fig_scatter.show()